<a href="https://colab.research.google.com/github/jitendra-manwani/GenAI-architect/blob/main/Jitendra_Assignment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install required packages
!pip install langchain langchain-community openai yfinance langfuse==2.17.0


**Step 1: Set Up API Keys & Imports**


In [ ]:
import os

# Azure OpenAI credentials
os.environ["AZURE_OPENAI_API_KEY"] = "63a0587ef7b24ca9b5d7b179c13fbaab"
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://eastus.api.cognitive.microsoft.com/"
os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"] = "telcogpt"

# Langfuse credentials
os.environ["LANGFUSE_PUBLIC_KEY"] = "pk-lf-a14e74c0-bb1e-4972-93ed-73ce9930d387"
os.environ["LANGFUSE_SECRET_KEY"] = "sk-lf-c6f9d29c-96cf-4bf4-82e0-0259bbfaee0d"
os.environ["LANGFUSE_HOST"] = "https://cloud.langfuse.com"

# Set OpenAI API base & type
os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["OPENAI_API_VERSION"] = "2024-12-01-preview"


**Step 2: Accept Company Name Input**

In [ ]:
# Step 1: Accept input
company_name = input("Enter the company name (e.g., Apple Inc): ")

Enter the company name (e.g., Apple Inc): Google


**Step 3: Extract Ticker Symbol from Yahoo Finance**

In [ ]:
import yfinance as yf

def get_ticker(company_name):
    # Static lookup fallback
    static_map = {
        "Apple Inc": "AAPL",
        "Microsoft Corporation": "MSFT",
        "Google": "GOOGL",
        "Amazon": "AMZN"
    }
    if company_name in static_map:
        return static_map[company_name]
    else:
        # Try yfinance search
        try:
            ticker = yf.Ticker(company_name).info['symbol']
            return ticker
        except:
            return None

ticker = get_ticker(company_name)
print("Ticker symbol:", ticker)


Ticker symbol: GOOGL


**Step 4: Fetch Company News from Yahoo Finance**

In [ ]:
# !pip install feedparser #one time installation of feedparser
import feedparser

def fetch_news_from_rss(ticker):
    feed_url = f"https://feeds.finance.yahoo.com/rss/2.0/headline?s={ticker}&region=US&lang=en-US"
    feed = feedparser.parse(feed_url)

    summaries = []
    for entry in feed.entries[:5]:  # Get top 5 entries
        title = entry.get('title', '')
        summary = entry.get('summary', '')
        summaries.append(f"{title} - {summary}")

    return summaries

news_summaries = fetch_news_from_rss(ticker)

if news_summaries:
    print("News Summaries:\n")
    for summary in news_summaries:
        print("-", summary)
else:
    print("No recent news found for this ticker.")


News Summaries:

- Potential Alphabet (GOOGL) and OpenAI Deal Could Shift Dynamics in AI Race - Alphabet Inc. (NASDAQ:GOOGL) is one of the 10 best tech stocks to buy according to billionaires right now. On Tuesday, June 10, Reuters reported that OpenAI is partnering with Google Cloud to meet its growing need for computing power, citing three sources familiar with the matter. As both these companies are seen as competitors in AI space, […]
- 2 Artificial Intelligence (AI) Cloud Stocks to Buy in June - Oracle expects its cloud revenue growth to accelerate to at least 40% over the next year.  Google Cloud is contributing a growing percentage of profits to Alphabet.  Artificial intelligence (AI) is providing profitable opportunities for businesses and investors.
- Oracle Stock (ORCL) Delivers Strong Quarter as Cloud and AI Strategy Pays Off - Oracle (ORCL) has just wrapped up its quarterly reporting period with impressive results. Revenue rose 11% year-over-year to $15.9 billion, beating e

**Step 5: Analyze Sentiment Using GPT-4o (via Azure OpenAI + LangChain)**

In [ ]:
from langchain.chat_models import AzureChatOpenAI
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.prompts import ChatPromptTemplate

# Define the structure of the output JSON
response_schemas = [
    ResponseSchema(name="company_name", description="Name of the company"),
    ResponseSchema(name="stock_code", description="Ticker symbol of the company"),
    ResponseSchema(name="newsdesc", description="Short summary of the news"),
    ResponseSchema(name="sentiment", description="Sentiment of the news: Positive, Negative or Neutral"),
    ResponseSchema(name="people_names", description="People mentioned in the news"),
    ResponseSchema(name="places_names", description="Places mentioned in the news"),
    ResponseSchema(name="other_companies_referred", description="Other companies mentioned"),
    ResponseSchema(name="related_industries", description="Related industries"),
    ResponseSchema(name="market_implications", description="What this means for the market"),
    ResponseSchema(name="confidence_score", description="Confidence in sentiment analysis, as a float from 0 to 1")
]

# Create the output parser
parser = StructuredOutputParser.from_response_schemas(response_schemas)

# Combine news summaries into a single input string
input_news = "\n\n".join(news_summaries)

# Define prompt template
format_instructions = parser.get_format_instructions().replace("{", "{{").replace("}", "}}")

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a financial news sentiment analyzer."),
    ("human", "{input_news}\n\nReturn result as JSON in this format:\n" + format_instructions)
])

# Azure OpenAI client using GPT-4o
llm = AzureChatOpenAI(
    deployment_name=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
    model="gpt-4o",
    temperature=0.3,
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"]
)

# Chain to analyze sentiment
chain = prompt | llm | parser

# Run the chain
response = chain.invoke({"input_news": input_news})
response['company_name'] = company_name
response['stock_code'] = ticker

# Print formatted output
import json
print(json.dumps(response, indent=2))


{
  "company_name": "Google",
  "stock_code": "GOOGL",
  "newsdesc": "Potential partnership between OpenAI and Google Cloud could alter competitive dynamics in the AI sector.",
  "sentiment": "Positive",
  "people_names": "",
  "places_names": "",
  "other_companies_referred": "OpenAI, Oracle",
  "related_industries": "Technology, Artificial Intelligence, Cloud Computing",
  "market_implications": "The partnership may strengthen Alphabet's position in the AI race and enhance its cloud service offerings.",
  "confidence_score": 0.85
}


**end of the code**